In [ ]:
#Code Below is by Veer Arora

import pandas as pd

In [ ]:
data1=pd.read_csv('modified_Page_1_39.csv')
data2=pd.read_csv('modified_Page_41_73.csv')
data3=pd.read_csv('modified_Page_75_140.csv')
data4=pd.read_csv('modified_Page_140_252.csv')
data5=pd.read_csv('modified_Page_253_X.csv')

data=[data1,data2,data3,data4,data5]

In [ ]:
data=[data1,data2,data3,data4,data5]
# drop columns containing the name 'Drop'
for df in data:
    drop_columns = [col for col in df.columns if 'Drop' in col]
    df.drop(columns=drop_columns, inplace=True)

data1, data2, data3, data4, data5 = data

## Fixing Data from Page 1-39

Combining Positive and Negative EBITDA values

In [ ]:
# function to convert values like (0.06) to negative numeric -0.06
def clean_negative_values(column):
    return column.replace(r'\((.*?)\)', r'-\1', regex=True).replace('None', None).astype(float)

# convertion
columns_to_clean = ['EBITDA Negative']
for column in columns_to_clean:
    data1[column] = clean_negative_values(data1[column])

# Create Ebitda from positive and negative
data1['EBITDA'] = data1['EBITDA Negative'].fillna(data1['EBITDA Positive'])

Droping the columns

In [ ]:
# drop cleanup columns
drop_columns1= ['EBITDA Negative','EBITDA Positive']
data1.drop(columns=drop_columns1, inplace=True)

## Fixing Data from Page 41-73

Fixing the Post Valuation Data

In [ ]:
# updating post valuation based on found pattern leading to data issues
data2['Post Valuation new'] = data2['Post Valuation 3'].combine_first(
    data2['Post Valuation 2 (use to Replace in other column) / Becomes Net Income if Replaced by Postvaluation 3']
).combine_first(
    data2['Post Valuation']
)

data2['Post Valuation']= data2['Post Valuation new']

Fixing Net Income Positive Column

In [ ]:
# updating Net Income based on found pattern leading to data issues
data2['Net Income Positive'] = data2.apply(
    lambda row: row['Post Valuation 2 (use to Replace in other column) / Becomes Net Income if Replaced by Postvaluation 3']
    if pd.notna(row['Post Valuation 3']) else None,
    axis=1
)

Fixing negative and positive EBITDA and Net Income Columns

In [ ]:
# convert to true negative numbers
columns_to_clean = ['EBITDA Negative', 'Net Income Negative']
for column in columns_to_clean:
    data2[column] = clean_negative_values(data2[column])

# update EBITDA and Net Income columns
data2['EBITDA'] = data2['EBITDA Negative'].fillna(data2['EBITDA Positive'])
data2['Net Income'] = data2['Net Income Negative'].fillna(data2['Net Income Positive'])

Droping and cleaning the Column Names

In [ ]:
#Making the column names uniform
data2['CEO Education']= data2['CEO Education / Sometimes DealType 2']
data2['Deal Type 2']= data2['Deal Type 2 / Sometime Education']

In [ ]:
# drop cleanup columns
drop_columns2= ['Post Valuation 2 (use to Replace in other column) / Becomes Net Income if Replaced by Postvaluation 3', 'EBITDA Negative', 'Net Income Negative', 'EBITDA Positive', 'Post Valuation 3',
'Net Income Positive','Post Valuation new', 'CEO Education / Sometimes DealType 2','Deal Type 2 / Sometime Education']
data2.drop(columns=drop_columns2, inplace=True)

## Fixing Data from Page 75-140

Convert EBITDA and Net Income which are all negative to true negaitves

In [ ]:
# convert to true negative numbers
columns_to_clean = ['EBITDA', 'Net Income']
for column in columns_to_clean:
    data3[column] = clean_negative_values(data3[column])

Fixing Columns with data in mulltiple other columns

In [ ]:
# Update Investors data by combining columns where the data may lie
data3['New Investors'] = data3['New Investors/ Investors'].fillna('') + ', ' + data3['Vertical (Sometimes), Investors Sometimes'].fillna('')
data3['New Investors'] = data3['New Investors'].str.strip(', ').replace('^,|,$', '', regex=True)

# Update Vertical data by combining columns where the data may lie
data3['Vertical'] = data3['Vertical / Sometimes Series'].fillna('') + ', ' + data3['Vertical (Sometimes), Investors Sometimes'].fillna('')
data3['Vertical'] = data3['Vertical'].str.strip(', ').replace('^,|,$', '', regex=True)

# Update Deal Type 2 data by combining columns where the data may lie
data3['Deal Type 2'] = data3['Deal Type 2'].fillna('') + ', ' + data3['Vertical / Sometimes Series'].fillna('')
data3['Deal Type 2'] = data3['Deal Type 2'].str.strip(', ').replace('^,|,$', '', regex=True)

Renam / Drop Columns

In [ ]:
data3.drop(columns=['New Investors/ Investors', 'Vertical (Sometimes), Investors Sometimes',
                    'Vertical / Sometimes Series','Current Financing Status'], inplace=True)

## Fixing Data from Page 140-252

Fixing the Post Valuation

In [ ]:
# Update Post Valuation column based on found pattern
data4['Post Valuation'] = data4['Replace Post Valuation with this cell value'].combine_first(
    data4['Post Valuation']
)


Convert EBITDA, Net Income, Gross Profit which are all negative to true negaitves

In [ ]:
# convert to true negative numbers
columns_to_clean = ['EBITDA negative', 'Net Income negative', 'Gross Profit negative']
for column in columns_to_clean:
    data4[column] = clean_negative_values(data4[column])

# Update EBITDA, Net Income and Gross PRofit with true negative numbers
data4['EBITDA'] = data4['EBITDA negative']
data4['Net Income'] = data4['Net Income negative']
data4['Gross Profit'] = data4['Gross Profit negative']

In [ ]:
data4.drop(columns=['EBITDA negative', 'Gross Profit negative',
       'Net Income negative', 'Replace Post Valuation with this cell value', 'Series'], inplace=True)


## Fixing Data from Page 253 - X

Convert EBITDA, Net Income which are all negative to true negaitves

In [ ]:
# convert to true negative numbers
columns_to_clean = ['EBITDA negative', 'Net Income negative']
for column in columns_to_clean:
    data5[column] = clean_negative_values(data5[column])

# Update EBITDA and Net Income columns
data5['EBITDA'] = data5['EBITDA negative']
data5['Net Income'] = data5['Net Income negative']

Creating Columns with data from multiple other columns

In [ ]:
# Deal Date
data5['Deal Date'] = data5['Vertical or Deal Date']

# combine the data from multiple columns for Investors data
data5['New Investors'] = data5['Vertical, Investor, Deal Type 2'].fillna('') + ', ' + data5['Can be Investor, Deal Type 2, Vertical'].fillna('')
data5['New Investors'] = data5['New Investors'].str.strip(', ').replace('^,|,$', '', regex=True)

# combine the data from multiple columns for Vertical data
data5['Vertical'] = data5['Vertical, Investor, Deal Type 2'].fillna('') + ', ' + data5['Can be Investor, Deal Type 2, Vertical'].fillna('')+ ', ' + data5['Vertical or Deal Date'].fillna('')
data5['Vertical'] = data5['Vertical'].str.strip(', ').replace('^,|,$', '', regex=True)

#combine the data from multiple columns for Deal Type 2 data
data5['Deal Type 2'] = data5['Vertical, Investor, Deal Type 2'].fillna('') + ', ' + data5['Can be Investor, Deal Type 2, Vertical'].fillna('')
data5['Deal Type 2'] = data5['Deal Type 2'].str.strip(', ').replace('^,|,$', '', regex=True)

Dropping unncessary Columns

In [ ]:
data5.drop(columns=['EBITDA negative', 'Net Income negative',
'Vertical or Deal Date',
'Can be Investor, Deal Type 2, Vertical',
'Vertical, Investor, Deal Type 2'], inplace=True)


## Saving the Data in new CSV files

In [ ]:
import os
# save data to CSV
folder_path = "/Users/veerarora/Desktop/INDENG 242A/Team Project/Pitchbook"
file1  = os.path.join(folder_path, f"Cleanup_Page_1_39.csv")
data1.to_csv(file1, index=False)
file2  = os.path.join(folder_path, f"Cleanup_Page_41_73.csv")
data2.to_csv(file2, index=False)
file3  = os.path.join(folder_path, f"Cleanup_Page_75_140.csv")
data3.to_csv(file3, index=False)
file4  = os.path.join(folder_path, f"Cleanup_Page_140_252.csv")
data4.to_csv(file4, index=False)
file5  = os.path.join(folder_path, f"Cleanup_Page_253_X.csv")
data5.to_csv(file5, index=False)